<a href="https://colab.research.google.com/github/Manjesh80/ml/blob/main/FineTune_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !rm -rf ./ml
# !git clone https://github.com/manjesh80/ml.git
# !rm -f ml/*.ipynb
# !cp ml/* .
# !ls

In [2]:
!wget https://raw.githubusercontent.com/Manjesh80/ml/main/sent_train.csv

--2024-05-07 19:48:03--  https://raw.githubusercontent.com/Manjesh80/ml/main/sent_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 858645 (839K) [text/plain]
Saving to: ‘sent_train.csv’

sent_train.csv      100%[===================>] 838.52K  --.-KB/s    in 0.04s   

2024-05-07 19:48:03 (22.8 MB/s) - ‘sent_train.csv’ saved [858645/858645]



In [3]:
import subprocess
import torch

def useGPU():
    if not torch.cuda.is_available():
        print(f" !!!! No GPU detected! This notebook will be slow !!!! \n\n")
    else:
        if torch.cuda.is_available():
          has_gpu()
          gpu_device = torch.device('cuda')
          device = torch.device('cuda')
          gpu_info = torch.cuda.get_device_properties(gpu_device)
          gpu_memory = gpu_info.total_memory / 1e9  # Convert bytes to gigabytes
          print(f"GPU: {gpu_info.name}, Total Memory: {gpu_memory:.2f} GB")
        else:
            device = torch.device("cpu")
            print("No GPU detected.")

def install_packages(packages):
    check = u'\u2705'
    print("\033[1mInstalling base requirements...\n\033[0m")
    for package in packages:
        process_scatter = subprocess.run(
            ["pip", "install", package],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )
        if process_scatter.returncode != 0:
            print(f"Installation of {package} failed with error:\n{process_scatter.stderr.decode('utf-8')}")
        else:
            print(f"{check} {package} installation completed successfully!\n")

def has_gpu():
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)

useGPU()
device =  "cuda" if torch.cuda.is_available() else "cpu"


Tue May  7 19:48:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   43C    P8              12W /  72W |      4MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
packages = [ "transformers[torch]" , "datasets" , "torchinfo" ,
            "evaluate" , "wget" , "tqdm" , "bitsandbytes" , "trl" ,
             "peft" , "torchvision" , "boto3" , "huggingface_hub", "accelerate"]

install_packages(packages)

Installing base requirements...

✅ transformers[torch] installation completed successfully!

✅ datasets installation completed successfully!

✅ torchinfo installation completed successfully!

✅ evaluate installation completed successfully!

✅ wget installation completed successfully!

✅ tqdm installation completed successfully!

✅ bitsandbytes installation completed successfully!

✅ trl installation completed successfully!

✅ peft installation completed successfully!

✅ torchvision installation completed successfully!

✅ boto3 installation completed successfully!

✅ huggingface_hub installation completed successfully!

✅ accelerate installation completed successfully!



In [21]:
from huggingface_hub import login
from getpass import getpass
access_token_read = getpass('Enter Huggingface read token: ')
login(token = access_token_read)

Enter Huggingface read token: ··········
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING
import datasets
from peft import PeftModel
from peft import AutoPeftModelForCausalLM


In [9]:
def evaluate(y_true, y_pred):
    mapping = {"positive": 2, "neutral": 1, "none": 1, "negative": 0}

    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f"Accuracy: {accuracy:.3f}")

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true)) if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f"Accuracy for label {label}: {accuracy:.3f}")

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print("\nClassification Report:")
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print("\nConfusion Matrix:")
    print(conf_matrix)


def inference(pipe, prompt):
    result = pipe(prompt)
    answer = result[0]["generated_text"].split("=")[-1]
    return answer


def predict(X_test, model, tokenizer):
    y_pred = []
    pipe = pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=1,
        temperature=0.01,
        # device='cuda'
    )
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        answer = inference(pipe, prompt)

        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")
    return y_pred


def prepare_data():
    filename = "sent_train.csv"
    df = pd.read_csv(filename)
    # df['text'] = df['input']
    df["sentiment"] = df["label"].map(
        {
            0: "negative",
            1: "positive",
            2: "neutral",
        }
    )

    X_train = list()
    X_test = list()
    for sentiment in ["positive", "neutral", "negative"]:
        train, test = train_test_split(
            df[df.sentiment == sentiment],
            train_size=300,
            test_size=300,
            random_state=42,
        )
        X_train.append(train)
        X_test.append(test)

    X_train = pd.concat(X_train).sample(frac=1, random_state=10)
    X_test = pd.concat(X_test)

    eval_idx = [
        idx for idx in df.index if idx not in list(train.index) + list(test.index)
    ]
    X_eval = df[df.index.isin(eval_idx)]
    X_eval = X_eval.groupby("sentiment", group_keys=False).apply(
        lambda x: x.sample(n=50, random_state=10, replace=True)
    )
    X_train = X_train.reset_index(drop=True)

    X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1), columns=["text"])
    X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1), columns=["text"])

    y_true = X_test.sentiment
    X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

    train_data = Dataset.from_pandas(X_train)
    eval_data = Dataset.from_pandas(X_eval)

    return train_data, eval_data, X_test, y_true


def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [{data_point["text"]}] = {data_point["sentiment"]}
            """.strip()


def generate_test_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [{data_point["text"]}] = """.strip()


def init_model(model_name="meta-llama/Meta-Llama-3-8B"):
    compute_dtype = getattr(torch, "float16")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        quantization_config=bnb_config,
        trust_remote_code=True,
    )
    model.config.use_cache = False
    model.config.pretraining_tp = 1

    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True,
    )
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    return model, tokenizer


def init_model_from_checkpoint(peft_model):
    model = AutoPeftModelForCausalLM.from_pretrained(
        peft_model, device_map="auto", torch_dtype=torch.float16
    )
    tokenizer = AutoTokenizer.from_pretrained(peft_model)
    return model, tokenizer


In [10]:
train_data, eval_data, X_test, y_true = prepare_data()

START TRAINING

In [11]:

model, tokenizer = init_model()

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    # target_modules=target_modules
)


training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,  # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    evaluation_strategy="epoch",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=1024,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [12]:
# Train model
trainer.train()

Epoch,Training Loss,Validation Loss
0,1.067400,1.076069
1,0.867700,1.030015


TrainOutput(global_step=224, training_loss=1.0871637633868627, metrics={'train_runtime': 1217.3029, 'train_samples_per_second': 1.479, 'train_steps_per_second': 0.184, 'total_flos': 6295989946834944.0, 'train_loss': 1.0871637633868627, 'epoch': 1.991111111111111})

In [13]:
# Save trained model
output_dir = "results/adapter_fine_tune_llama3"
trainer.save_model(output_dir)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# SAVE MODEL TO LOCAL DIRECTORY



In [15]:
model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)
# Merge LoRA and base model and save
merged_model = model.merge_and_unload()

merged_dir = "results/final_fine_tune_llama3"

merged_model.save_pretrained(merged_dir, safe_serialization=True, max_shard_size="2GB")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
# Empty VRAM
# del model
# del merged_model
# del trainer
# del tokenizer
import gc
gc.collect()
gc.collect()

0

USE SAVED MODEL

In [20]:
model = AutoPeftModelForCausalLM.from_pretrained(
  merged_dir,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(merged_dir)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-663a8cc8-049635921f14181a4e636106;a712723f-7dee-46b2-9ab0-b7a986f73f93)

Repository Not Found for url: https://huggingface.co/results/fine_tune_llama3_final/resolve/main/adapter_model.bin.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

In [ ]:
y_pred = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred)